# Part 8 - Deep Learning (DL)

![Alt](./deeplearning.png "Deep Learning")

### Neuron
In DL a neuron is a node that processes the weighted inputs and passes the signal onto another neuron along the line.

### Activation function
* Threshold $ \phi(x) = \text{0 if x < 0, 1 otherwise} $

* Sigmoid $ \phi(x) = \frac{1}{1+e^{-x}} $
    - Smooth transition from 0 to 1
    - good for predicting probabilities in the final/outpit layer
    
* Rectifier (Relu?) $ \phi(x) = max(x,0) $
    - _[Deep sparse rectifier neural networks](http://jmlr.org/proceedings/papers/v15/glorot11a/glorot11a.pdf) by Xavier Glorot et al. 2011_

* Hyperbolic Tangent (tanh) $ \phi(x) = \frac{1-e^{-2x}}{1+e^{-2x}} $

__Cost function__: $C = 0.5(\hat{y}-y)^2$, function to be minimized

[A list of cost functions used in neural network, alongside applications](http://stats.stackexchange.com/questions/154879/a-list-of-cost-functions-used-in-neural-network-alongside-applications) CrossValidated 2015

### Gradient Descent
* finding the right weights that give the minimum cost function requires a multi-demensional optimization problem
* Convex cost function
* Batch gradient descent, where all the samples are used for the calculation of the cost function the then all the weights are adjusted

### Stochastic Gradient Descent
* Useful for cases when the cost function has local minimums, in oder to find the __global minimum__.
* The weights are adjust after each indivisual sample evaluation

[A Neural Network in 13 lines of Python (part 2 - gradient descent)](https://iamtrask.github.io/2015/07/27/python-network-part2/)

[Neural Network and Deep Learning by Michael Nielsen (2015)](http://neuralnetworksanddeeplearning.com/chap2.html)

### Backpropagation
* information propagates from output to input, allowing the update of all the weights simultanuously
* allow us to see which part of the network is responsible for the most error

## Artificial Neural Network (ANN) with Stochastic Gradient Descent

__Step 1__: Randomly initialize the weights to small numbers close to 0, but not 0.

__Step 2__: Input the first observation of you dataset in the input layer, each feature in one input node.

__Step 3__: Forward-Propagation: from left to right, the neurons are activated in a way that the impact of each neuron's activation is limited by the weights. Propagate the activations until getting the predicted result y.

__Step 4__: Compare the predicted result to tue actual result. Measure the generated error.

__Step 5__: Back-Porpagation: from right to left, the error is back propagated. Update the weights according to how much they are responsible for the error. The learning rate decides by how much we update the weights.

__Step 6__: Repeat steps 1-5 and update the weights after each observation (Reinforcement Learning). Or: Repeat steps 1-5 but update the weights only after a batch of observation (Batch Learning)

__Step 7__: When the whole training set passed throught the ANN, that makes and epoch. Redo more epochs.

In [1]:
# import the libraries that will be used
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

from keras.preprocessing.image import ImageDataGenerator


/home/sousae/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
# import dataset
dir1 = '/home/sousae/Classes/udemy_machineLearning_A-Z/Part8_Deep_Learning/'
dataset = pd.read_csv(dir1+'Churn_Modelling.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [22]:
# Preprocess the data

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

X[0,:]

array([619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

In [23]:
# Encoding categorical data
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1,2])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
X[0,:]

array([0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
       6.1900000e+02, 4.2000000e+01, 2.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0134888e+05])

In [24]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
X_train[1,:]

array([ 1.75486502, -0.57369368, -0.91601335,  0.91601335, -2.30455945,
        0.30102557, -1.37744033, -0.00631193, -0.92159124,  0.64259497,
        0.9687384 , -0.74866447])

In [26]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 12))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

/home/sousae/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=12, units=6, kernel_initializer="uniform")`
  """
/home/sousae/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
/home/sousae/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  # This is added back by InteractiveShellApp.init_path()
/home/sousae/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
8000/8000 [==============================] - 8s 1ms/step - loss: 0.5577 - acc: 0.7957
Epoch 2/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.5082 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5061 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.5059 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 7s 927us/step - loss: 0.5059 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5059 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5059 - acc: 0.7960
Epoch 8/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5058 - acc: 0.7960
Epoch 9/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5059 - acc: 0.7960
Epoch 10/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5058 - acc: 0.7960
Epoch 11/100
80

8000/8000 [==============================] - 8s 1ms/step - loss: 0.3543 - acc: 0.8516
Epoch 84/100
8000/8000 [==============================] - 8s 1ms/step - loss: 0.3534 - acc: 0.8537
Epoch 85/100
8000/8000 [==============================] - 7s 920us/step - loss: 0.3539 - acc: 0.8515
Epoch 86/100
8000/8000 [==============================] - 7s 872us/step - loss: 0.3534 - acc: 0.8532
Epoch 87/100
8000/8000 [==============================] - 8s 1ms/step - loss: 0.3533 - acc: 0.8541
Epoch 88/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.3529 - acc: 0.8526
Epoch 89/100
8000/8000 [==============================] - 7s 903us/step - loss: 0.3527 - acc: 0.8540
Epoch 90/100
8000/8000 [==============================] - 8s 1ms/step - loss: 0.3520 - acc: 0.8541
Epoch 91/100
8000/8000 [==============================] - 8s 971us/step - loss: 0.3513 - acc: 0.8535
Epoch 92/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.3510 - acc: 0.8551
Epoch 93/100


In [5]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1507   88]
 [ 185  220]]


## Convolutional Neural Network

__Step 1: Convolution__
$$ (f*g)(t) = \int_{-\infty}^{\infty} f(\tau)g(t-\tau) d\tau $$

[Introduction to Convolutional Neural Networks by Jianxin Wu](http://cs.nju.edu.cn/wujx/paper/CNN.pdf)

[Understanding Convulutional Neural Networks with a Mathematical Model by C.C. Jay Kup 2016](https://arxiv.org/pdf/1609.04112.pdf)

Input image + Feature Detector matrix (3x3) = Feature Map
* The process will make the picture smaller.
* Finds the features of the image while keep the spacial relations between

There's multiple feature maps created using different filters/feature detector.

__Step 1 (B) is a ReLU layer to beak up the linearity.__


__Step 2: Max Pooling__
* Used to address spacial variance (i.e. tild, stretch, distortion in an image)
* Feature Map ---- Max pooling ----> Pooled Feature Map
* Overlap with a matrix and selecting the maximun
* Preserves the fearures
* Reduces the size of the parameter space
* Prevents overfitting by disregarding unnecessary information

[Evaluation of Pooling Operations in Convolutional Architectures for Object Recognition by Dominik Scherer et al. 2010](http://ais.uni-bonn.de/papers/icann2010_maxpool.pdf)


__Step 3: Flattening__
* Pooled Feature Map ---- Flattening ----> vector to be the input to an ANN


__Step 4: Full Connection__
* Add an ANN to the previous steps

[The 9 Deep learning papers You need to know about by Adit Deshpande 2016](https://adeshpande3.github.io/adeshpande3.github.io/The-9-Deep-Learning-Papers-You-Need-To-Know-About.html)

#### Softmax and Cross-entropy
* Softmax function $ f_j(z) = \frac{e^{zj}}{\sum_k e^{zk}} $
* Makes sure the probabilities of the different classifications all add up to 1
* Cross-Entropy function, used to calculate the error in the classification instead of MSE
$$ L_i = \log \left(\frac{e^{f_{yi}}}{\sum_k e^{z_jk}} \right) $$
$$H(p,q) = -\sum_x p(x) \log q(x)$$

[A Friendly Introduction to Cross-Entropy Loss by Rob DiPietro 2016](https://rdipietro.github.io/friendly-intro-to-cross-entropy-loss/)

In [11]:
# Part 1 - Building the CNN

# Initialising the CNN
classifier = Sequential()

In [12]:
help(Convolution2D)

Help on class Conv2D in module keras.layers.convolutional:

class Conv2D(_Conv)
 |  2D convolution layer (e.g. spatial convolution over images).
 |  
 |  This layer creates a convolution kernel that is convolved
 |  with the layer input to produce a tensor of
 |  outputs. If `use_bias` is True,
 |  a bias vector is created and added to the outputs. Finally, if
 |  `activation` is not `None`, it is applied to the outputs as well.
 |  
 |  When using this layer as the first layer in a model,
 |  provide the keyword argument `input_shape`
 |  (tuple of integers, does not include the sample axis),
 |  e.g. `input_shape=(128, 128, 3)` for 128x128 RGB pictures
 |  in `data_format="channels_last"`.
 |  
 |  # Arguments
 |      filters: Integer, the dimensionality of the output space
 |          (i.e. the number output of filters in the convolution).
 |      kernel_size: An integer or tuple/list of 2 integers, specifying the
 |          width and height of the 2D convolution window.
 |        

In [13]:
# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape=(64, 64, 3), activation='relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(output_dim = 128, activation='relu'))
classifier.add(Dense(output_dim = 1, activation='sigmoid'))

# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3...)`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=128, activation="relu")`
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid")`
  app.launch_new_instance()


In [14]:
# Part 2 - Fitting the CNN to the images
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/disk1/sousae/Classes/udemy_machineLearning/Machine_Learning_A-Z/Part8_Deep_Learning/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/disk1/sousae/Classes/udemy_machineLearning/Machine_Learning_A-Z/Part8_Deep_Learning/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 25,
                         validation_data = test_set,
                         nb_val_samples = 2000)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=250, epochs=25, validation_steps=2000)`


Epoch 1/25
250/250 [==============================] - 198s 792ms/step - loss: 0.6759 - acc: 0.5635 - val_loss: 0.6048 - val_acc: 0.6708
Epoch 2/25
250/250 [==============================] - 149s 594ms/step - loss: 0.6099 - acc: 0.6703 - val_loss: 0.5849 - val_acc: 0.6852
Epoch 3/25
250/250 [==============================] - 148s 590ms/step - loss: 0.5549 - acc: 0.7170 - val_loss: 0.5350 - val_acc: 0.7403
Epoch 4/25
250/250 [==============================] - 147s 588ms/step - loss: 0.5259 - acc: 0.7318 - val_loss: 0.5327 - val_acc: 0.7313
Epoch 5/25
250/250 [==============================] - 148s 592ms/step - loss: 0.5066 - acc: 0.7483 - val_loss: 0.5023 - val_acc: 0.7509
Epoch 6/25
250/250 [==============================] - 149s 597ms/step - loss: 0.4898 - acc: 0.7598 - val_loss: 0.4910 - val_acc: 0.7660
Epoch 7/25
250/250 [==============================] - 148s 594ms/step - loss: 0.4798 - acc: 0.7671 - val_loss: 0.4824 - val_acc: 0.7700
Epoch 8/25
250/250 [============================